# Collaborative Filtering

- 콘텐츠 기반 필터링은 사용자에게 취향이 비슷한 다른 사용자들이 좋아하는 아이템을 추천해주는 장식입니다.
- 대표적인 방식은 User-Item Matrix를 Factorization하여 얻은 사용자와 아이템 임베딩의 내적을 통해 사용자의 아이템에 대한 선호도를 계산합니다/

In [32]:
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares 

In [33]:
# 1. CSV 파일 불러오기
mangoplate_df = pd.read_csv("./data/MangoPlate_CF.csv")
mangoplate_df.head()

,res_name,user_name,rating
0,서울치킨,박윤선,5
1,서울치킨,뭐든 잘 먹음,1
2,서울치킨,혜자,5
3,서울치킨,우 리,5
4,서울치킨,GoodVibesOnly,3


In [34]:
# 2. User-Item Matrix 만들기
user_list = list(set(mangoplate_df['user_name'].tolist()))
item_list = list(set(mangoplate_df['res_name'].tolist()))

user2idx = {k: v for v, k in enumerate(user_list)}
item2idx = {k: v for v, k in enumerate(item_list)}

idx2user = {k: v for k, v in enumerate(user_list)}
idx2item = {k: v for k, v in enumerate(item_list)}

data = mangoplate_df["rating"].tolist()
row = mangoplate_df["user_name"].apply(lambda x: user2idx.get(x))
col = mangoplate_df["res_name"].apply(lambda x: item2idx.get(x))

matrix = csr_matrix((data, (row, col)))
matrix.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 5, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 3, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [35]:
# 3. Matrix Factorization Model - ALS 학습
model = AlternatingLeastSquares(factors=20)
model.fit(matrix)

100%|██████████| 15/15 [00:00<00:00, 20.52it/s]


In [36]:
# 4. 추천 예시
for i in range(3):
    user_idx = random.randint(0, len(user2idx))
    print(f"Recommendation for User {idx2user[user_idx]}:")
    
    print(f"We can recommend following restaurants to user {idx2user[user_idx]}:")
    recommend_items_idx, _ = model.recommend(user_idx, matrix[user_idx])
    for item_idx in recommend_items_idx[:5]:
        print(f"\t{idx2item[item_idx]}") 

Recommendation for User 난!신나게살거야...:
We can recommend following restaurants to user 난!신나게살거야...:
	떡반집(둔산점)
	원조태평소국밥
	대선칼국수
	서울치킨
	동방커피
Recommendation for User 사과맛다시:
We can recommend following restaurants to user 사과맛다시:
	성심당케익부띠끄
	Fake
	팬텀팬스테이크
	몇몇커피
	누오보나폴리
Recommendation for User 송영채:
We can recommend following restaurants to user 송영채:
	하프레스트
	무이
	메떼레
	성심당케익부띠끄
	동방커피
